# AUV 2D

In [ ]:
import sys
sys.path.append("../src")
from imperium.dynamics.symbolic import System
from sympy import *
init_printing()

In [ ]:
# state
x, y, vx, vy = Matrix(symbols('x y v_x v_y', real=True))
state = Matrix([x, y, vx, vy])

# controls
ut, ux, uy = symbols('u_t', real=True, nonnegative=True), *symbols('u_x u_y', real=True)
control = Matrix([ut, ux, uy])

# constants
CD, m, A, T, rho = symbols('C_D m A T rho', real=True, nonnegative=True)
params = Matrix([m, T])

# state dynamics
dynamics = Matrix([
    vx,
    vy,
    T*ut*ux/m,
    T*ut*uy/m
])

# homotopy parameter
alpha = symbols('alpha', real=True, nonnegative=True)

# Lagrangian
lagrangian = alpha*ut + (1 - alpha)*ut**2

# equality constraints
equality = Matrix([sqrt(ux**2 + uy**2) - 1])

# inequality constraints
inequality = Matrix([ut-1, -ut, ux-1, -1-ux, uy-1, -1-uy])

# define nondimensional parameters
tb, lb, mb = symbols('t_b l_b m_b')

# dimensional transformations
units = Matrix([lb, lb, lb/tb, lb/tb, 1, 1, 1, mb*lb/tb, mb, 1])

In [ ]:
# instantiate the system
sys = System(state, control, dynamics, lagrangian, units)
sys.dfs

In [ ]:
sys.ndunits

In [ ]:
# optimal thrust direction (solved by hand with geometric method)
utheta = -Matrix([sys.l[2], sys.l[3]])/sqrt(sys.l[2]**2 + sys.l[3]**2)
utheta.T

In [ ]:
# gradient of Hamiltonian
sys.KKT([(ux, utheta[0]), (uy, utheta[1])])
sys.KKTeqs

In [ ]:
# solving
sol = solve(sys.KKTeqs, sys.KKTvars, simplify=True, force=True)

In [ ]:
# now assign optimal controls
sys.uo = Matrix([[sol[ut]], utheta])
sys.uo.T

In [ ]:
# define nondimensional parameters
tb, lb, mb = symbols('t_b l_b m_b')

In [ ]:
# dimensionalising units
units = Matrix([lb, lb, lb/tb, lb/tb, 1, 1, 1, mb*lb/tb, mb, 1])
units.T

In [ ]:
sys.ndim

In [ ]:
# make some fortran code!
sys.codegen("../src/imperium/dynamics/auv2d", "F")

# Using fortran code

In [1]:
import sys
sys.path.append("../src")
from imperium.dynamics import auv2d
from imperium.segment import Indirect
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# instantiate segment
seg = Indirect(auv2d, [0.7, 10, 10, 0.3, 1.225], [0, -1, -1], [1, 1, 1])

In [3]:
# set boundaries
seg.set(0, np.array([0, 0, 1, 1]), 1000, np.array([5, 5, 0, 0]))

In [ ]:
seg.propagate(np.random.random(4)*100, [0], True)

In [ ]:
plt.plot(seg.times, seg.states[:, 5])
plt.show()

In [ ]:
seg.states

In [11]:
auv2d.dimensionalise(10, 10, 5.2, 5.1, .3, .5, .5, 10, 10, 1, 10, 10, 10)

array([[100. ],
       [100. ],
       [  5.2],
       [  5.1],
       [  0.3],
       [  0.5],
       [  0.5],
       [100. ],
       [100. ],
       [  1. ]])